A título de curiosidade, estou incluíndo o modeloperiquitov11.ipynb, pela simples razão que foi impossível rodar aquele código com minhas configurações atuais, mas talvez teria um resultado bem interessante

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pnd



### Adicionando csv e label encoder

In [12]:
encoder = LabelEncoder()
dadosDeTreino = pnd.read_csv('train.csv')
dadosDeTreino.dropna(inplace=True)
encoder.fit(dadosDeTreino['explicit'])
dadosDeTreino['explicit'] = encoder.transform(dadosDeTreino['explicit'])
encoder.fit(dadosDeTreino['track_genre'])
dadosDeTreino['track_genre'] = encoder.transform(dadosDeTreino['track_genre'])

### Buscando correlações entre as variáveis

In [14]:
dadosDeTreino[[ 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'popularity_target']].corr()['popularity_target'].sort_values(ascending=False)

popularity_target    1.000000
danceability         0.057848
track_genre          0.054439
loudness             0.048140
time_signature       0.036454
explicit             0.016312
tempo                0.012839
key                  0.004285
liveness            -0.007691
acousticness        -0.009095
mode                -0.013869
energy              -0.025979
valence             -0.043138
speechiness         -0.091055
instrumentalness    -0.100967
Name: popularity_target, dtype: float64

#### Hipóteses
&nbsp;&nbsp;&nbsp;&nbsp; A princípio, o instrumentalness é a feature que mais afeta a popularidade, mas de forma negativa. Em muitos casos, as músicas com vocais tendem a ser mais populares por diversas questões, como por exemplo a facilidade de lembrar de uma música pela letra ou a conexão mais profunda com o conteúdo. <br>
&nbsp;&nbsp;&nbsp;&nbsp; A segunda maior correlação também é inversamente proporcional, e da mesma forma, se refere à presença de palavras faladas na música. Ironicamente, parece que a presença de vocais na música é muito importante, mas quanto menos palavras tiverem melhor.<br>
&nbsp;&nbsp;&nbsp;&nbsp; Por fim, vemos que a nota em que a música está não parece fazer uma diferença significativa na popularidade da música, talvez porque realmente não seja uma coisa que o ouvinte médio consiga até perceber entre uma música e outra.

### Criando conjunto de treino
#### Mesmo com a análise acima, adicionar todas as colunas como features melhorou o resultado razoavelmente as custas de tempo de processamento

In [7]:
x=dadosDeTreino[[ 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']]
y=dadosDeTreino['popularity_target']
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.1, random_state=42)

&nbsp;&nbsp;&nbsp;&nbsp; Para encontrar os parâmetros abaixo de cada modelo, diversos GridSearchCV foram realizados, levando entre 5-13h cada. Não foi divertido. Estarei incluíndo um dos exemplos aqui, mas não recomendo tentar rodá-los. Alguns outros estão perdidos entre um total de 11 arquivos em que foram feitos diversos testes. 

In [ ]:
classifier = RandomForestClassifier()
grid_parametros = {
    'n_jobs': [-1],
    'n_estimators': [100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 9, 11, 13, 15, 20, 50, 100],
    'max_features': [None, 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'random_state': [42],
}
melhorClassificador = GridSearchCV(classifier, grid_parametros, cv=5,scoring='accuracy' ,n_jobs=-1)
melhorClassificador.fit(x_treino, y_treino)
print('****************************************************************************************')
print(melhorClassificador.best_params_)

In [8]:
modelos_base = [('modelo9', ExtraTreesClassifier(criterion='gini', max_depth=None, max_features=None, n_estimators=1000, n_jobs=-1)),('modelo8', RandomForestClassifier(bootstrap=True, criterion='gini',  max_depth=None, max_features=None, n_estimators=1000, n_jobs=-1, random_state=42)),('modelo2',RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=20, max_features=None, n_estimators=500, n_jobs=-1, random_state=42))]
meta_modelo = SVC()
modelo_stack = StackingClassifier(estimators=modelos_base, final_estimator=meta_modelo, cv=5)
modelo_stack.fit(x_treino, y_treino)
print("*******************************************************")
print(accuracy_score(y_teste, modelo_stack.predict(x_teste)))
print(classification_report(y_teste, modelo_stack.predict(x_teste)))

print("*******************************************************")

*******************************************************
0.8300751879699249
              precision    recall  f1-score   support

           0       0.84      0.83      0.83      4092
           1       0.82      0.83      0.83      3888

    accuracy                           0.83      7980
   macro avg       0.83      0.83      0.83      7980
weighted avg       0.83      0.83      0.83      7980

*******************************************************


In [4]:
df = pnd.read_csv('test.csv')
df.dropna(inplace=True)
encoder.fit(df['explicit'])
df['explicit'] = encoder.transform(df['explicit'])
encoder.fit(df['track_genre'])
df['track_genre'] = encoder.transform(df['track_genre'])
x=df[[ 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']]
df['popularity_target'] = modelo_stack.predict(x)
df[['track_unique_id', 'popularity_target']].to_csv('submission7.csv', index=False)